In [1]:
# Import Dependencies and set variables
import pandas as pd
from pycoingecko import CoinGeckoAPI
from datetime import datetime
from_date = datetime(2015, 1, 1)
cg = CoinGeckoAPI()
currency = 'usd'

In [2]:
# Gather coin list
coins_list = cg.get_coins_list()
coins_list
for coin in coins_list:
    if 'bitcoin' in coin.values():
        print(coin)

{'id': 'bitcoin', 'symbol': 'btc', 'name': 'Bitcoin'}


In [3]:
# Convert Coins list into dataframe
coins_df = pd.DataFrame(coins_list).set_index('id')
coins_df

,symbol,name
id,,
01coin,zoc,01coin
0-5x-long-algorand-token,algohalf,0.5X Long Algorand Token
0-5x-long-altcoin-index-token,althalf,0.5X Long Altcoin Index Token
0-5x-long-balancer-token,balhalf,0.5X Long Balancer Token
0-5x-long-bitcoin-cash-token,bchhalf,0.5X Long Bitcoin Cash Token
...,...,...
zyro,zyro,Zyro
zytara-dollar,zusd,Zytara Dollar
zyx,zyx,ZYX


In [4]:
# Gather list of exchanges
exchanges_list = cg.get_exchanges_list()
exchanges_list

[{'id': 'binance',
  'name': 'Binance',
  'year_established': 2017,
  'country': 'Cayman Islands',
  'description': '',
  'url': 'https://www.binance.com/',
  'image': 'https://assets.coingecko.com/markets/images/52/small/binance.jpg?1519353250',
  'has_trading_incentive': False,
  'trust_score': 10,
  'trust_score_rank': 1,
  'trade_volume_24h_btc': 389872.0370886583,
  'trade_volume_24h_btc_normalized': 389872.0370886583},
 {'id': 'gdax',
  'name': 'Coinbase Exchange',
  'year_established': 2012,
  'country': 'United States',
  'description': '',
  'url': 'https://www.coinbase.com',
  'image': 'https://assets.coingecko.com/markets/images/23/small/Coinbase_Coin_Primary.png?1621471875',
  'has_trading_incentive': False,
  'trust_score': 10,
  'trust_score_rank': 2,
  'trade_volume_24h_btc': 80023.48298302613,
  'trade_volume_24h_btc_normalized': 80023.48298302613},
 {'id': 'huobi',
  'name': 'Huobi Global',
  'year_established': 2013,
  'country': 'Seychelles',
  'description': '',
  '

In [5]:
# Convert exchanges list into dataframe
exchanges_df = pd.DataFrame(exchanges_list).drop(columns=['description', 'url', 'image', 'has_trading_incentive',
                                        'trust_score', 'trust_score_rank', 'trade_volume_24h_btc_normalized']).set_index('id')
exchanges_df

,name,year_established,country,trade_volume_24h_btc
id,,,,
binance,Binance,2017.0,Cayman Islands,389872.037089
gdax,Coinbase Exchange,2012.0,United States,80023.482983
huobi,Huobi Global,2013.0,Seychelles,59576.977062
kucoin,KuCoin,2014.0,Seychelles,43133.028118
crypto_com,Crypto.com Exchange,2019.0,Cayman Islands,38213.940920
...,...,...,...,...
phemex,Phemex,2019.0,Singapore,8189.329133
catex,Catex,2018.0,China,7266.288420
traderjoe,Trader Joe,NaN,None,6041.087477


In [6]:
# Get OHLC data
ohlc_data = cg.get_coin_ohlc_by_id(id = 'bitcoin', vs_currency = currency, days = 'max')
ohlc_data

[[1367280000000, 135.3, 141.96, 135.3, 135.3],
 [1367539200000, 117.0, 117.0, 91.01, 91.01],
 [1367884800000, 111.25, 118.33, 106.4, 106.4],
 [1368230400000, 112.64, 118.78, 112.64, 113.01],
 [1368576000000, 114.71, 117.18, 114.16, 114.16],
 [1368921600000, 115.5, 123.88, 115.5, 120.5],
 [1369267200000, 122.58, 125.75, 122.58, 125.75],
 [1369612800000, 131.7, 134.6, 128.99, 128.99],
 [1369958400000, 129.18, 132.13, 127.4, 127.98],
 [1370217600000, 129.09, 129.09, 121.16, 121.16],
 [1370563200000, 121.31, 121.71, 119.0, 119.0],
 [1370908800000, 110.09, 110.09, 99.75, 105.99],
 [1371254400000, 108.9, 108.9, 98.4, 98.4],
 [1371600000000, 99.99, 106.55, 99.99, 106.55],
 [1371945600000, 107.9, 110.91, 107.25, 107.25],
 [1372291200000, 108.0, 108.0, 102.44, 103.39],
 [1372550400000, 99.66, 99.66, 93.33, 95.39],
 [1372809600000, 97.5, 97.5, 89.49, 90.8],
 [1373155200000, 78.8, 78.8, 67.81, 68.08],
 [1373500800000, 77.5, 85.75, 76.39, 85.75],
 [1373846400000, 93.8, 95.58, 88.06, 94.0],
 [13741

In [7]:
# Format OHLC Data as DataFrame
ohlc_df = pd.DataFrame(ohlc_data, columns = ['date', 'open', 'high', 'low', 'close'])
ohlc_df['date'] = ohlc_df['date'].apply(lambda x: int(str(x)[:-3]))
ohlc_df['date'] = ohlc_df['date'].apply(lambda d: datetime.fromtimestamp(int(d)).strftime('%Y-%m-%d'))
ohlc_df['coin_id'] = 'bitcoin'
ohlc_df['date'] = pd.to_datetime(ohlc_df.date)
ohlc_df = ohlc_df.loc[ohlc_df['date'] >= from_date]
ohlc_df = ohlc_df.set_index('date')
ohlc_df

,open,high,low,close,coin_id
date,,,,,
2015-01-02,313.99,314.45,286.57,286.57,bitcoin
2015-01-06,260.94,295.87,260.94,295.87,bitcoin
2015-01-10,284.45,289.57,267.18,267.18,bitcoin
2015-01-14,268.35,268.35,172.15,210.56,bitcoin
2015-01-18,205.82,214.55,199.49,214.55,bitcoin
...,...,...,...,...,...
2021-10-10,53894.60,55125.46,53894.60,54711.87,bitcoin
2021-10-14,57572.51,57572.51,56335.04,57452.43,bitcoin
2021-10-18,61809.02,62017.97,61165.64,62017.97,bitcoin


In [16]:
# Get Historical Market Data
market_data = cg.get_coin_market_chart_by_id(id = 'bitcoin', vs_currency = currency, days = 'max')
market_data

{'prices': [[1367107200000, 135.3],
  [1367193600000, 141.96],
  [1367280000000, 135.3],
  [1367366400000, 117.0],
  [1367452800000, 103.43],
  [1367539200000, 91.01],
  [1367625600000, 111.25],
  [1367712000000, 116.79],
  [1367798400000, 118.33],
  [1367884800000, 106.4],
  [1367971200000, 112.64],
  [1368057600000, 113.0],
  [1368144000000, 118.78],
  [1368230400000, 113.01],
  [1368316800000, 114.713],
  [1368403200000, 117.18],
  [1368489600000, 114.5],
  [1368576000000, 114.156],
  [1368662400000, 115.5],
  [1368748800000, 123.1],
  [1368835200000, 123.88],
  [1368921600000, 120.501],
  [1369008000000, 122.58],
  [1369094400000, 122.9],
  [1369180800000, 123.0],
  [1369267200000, 125.748],
  [1369353600000, 131.7],
  [1369440000000, 130.77],
  [1369526400000, 134.6],
  [1369612800000, 128.985],
  [1369699200000, 129.179],
  [1369785600000, 132.13],
  [1369872000000, 127.401],
  [1369958400000, 127.98],
  [1370044800000, 129.09],
  [1370131200000, 121.3],
  [1370217600000, 121.16]

In [17]:
# Clean Up the Data
columns = ['prices', 'market_caps', 'total_volumes']
n = 0
timestamp_list = []
for column in columns:
    temp_list = market_data[column]
    column_list = []
    for item in temp_list:
        timestamp_list.append(str(item.pop(0)))
        column_list.append(item.pop())
    
    market_data[column] = column_list
    if n == 0:
        market_data['date'] = [x[:-3] for x in timestamp_list]
        n+=1

In [18]:
market_df = pd.DataFrame(market_data)
market_df['date'] = market_df['date'].apply(lambda d: datetime.fromtimestamp(int(d)).strftime('%Y-%m-%d'))
market_df['date'] = pd.to_datetime(market_df.date)
market_df = market_df.loc[market_df['date'] >= from_date]
market_df = market_df.set_index('date')
market_df['coin_id'] = 'bitcoin'
market_df

,prices,market_caps,total_volumes,coin_id
date,,,,
2015-01-01,314.446000,4.301448e+09,3.885591e+07,bitcoin
2015-01-02,286.572000,3.921358e+09,1.187789e+08,bitcoin
2015-01-03,260.936000,3.571640e+09,2.055001e+08,bitcoin
2015-01-04,273.220000,3.740880e+09,1.550381e+08,bitcoin
2015-01-05,285.373800,3.908408e+09,9.700290e+07,bitcoin
...,...,...,...,...
2021-10-21,62541.086933,1.178927e+12,4.870996e+10,bitcoin
2021-10-22,61029.270932,1.151509e+12,4.076646e+10,bitcoin
2021-10-23,61572.140434,1.161166e+12,2.850504e+10,bitcoin


In [22]:
ex_volume_data = cg.get_exchanges_by_id(id = 'kucoin', days = 365)
ex_volume_data

{'name': 'KuCoin',
 'year_established': 2014,
 'country': 'Seychelles',
 'description': '',
 'url': 'https://www.kucoin.com/',
 'image': 'https://assets.coingecko.com/markets/images/61/small/kucoin.jpg?1591610627',
 'facebook_url': 'https://www.facebook.com/kucoinexchangepage',
 'reddit_url': 'https://www.reddit.com/r/kucoin',
 'telegram_url': 'https://t.me/joinchat/FMqUpg1_vED9MSDJPPcFkg',
 'slack_url': '',
 'other_url_1': 'https://t.me/joinchat/FMqUpg0PI9WjzplzfYahjw',
 'other_url_2': '',
 'twitter_handle': 'kucoincom',
 'has_trading_incentive': False,
 'centralized': True,
 'public_notice': '',
 'alert_notice': '',
 'trust_score': 10,
 'trust_score_rank': 4,
 'trade_volume_24h_btc': 43013.27758137183,
 'trade_volume_24h_btc_normalized': 43013.27758137183,
 'tickers': [{'base': 'BTC',
   'target': 'USDT',
   'market': {'name': 'KuCoin',
    'identifier': 'kucoin',
    'has_trading_incentive': False},
   'last': 62900.7,
   'volume': 6113.81556444,
   'converted_last': {'btc': 0.99842

In [26]:
ticker_data = cg.get_exchanges_tickers_by_id(id = 'kraken')
ticker_data

{'name': 'Kraken',
 'tickers': [{'base': 'USDT',
   'target': 'USD',
   'market': {'name': 'Kraken',
    'identifier': 'kraken',
    'has_trading_incentive': False},
   'last': 1.0,
   'volume': 161437360.83216146,
   'converted_last': {'btc': 1.584e-05, 'eth': 0.00023635, 'usd': 1.0},
   'converted_volume': {'btc': 2557, 'eth': 38156, 'usd': 161437361},
   'trust_score': 'green',
   'bid_ask_spread_percentage': 0.02,
   'timestamp': '2021-10-26T08:48:23+00:00',
   'last_traded_at': '2021-10-26T08:48:23+00:00',
   'last_fetch_at': '2021-10-26T08:48:23+00:00',
   'is_anomaly': False,
   'is_stale': False,
   'trade_url': 'https://trade.kraken.com/markets/kraken/usdt/usd',
   'token_info_url': None,
   'coin_id': 'tether'},
  {'base': 'USDT',
   'target': 'EUR',
   'market': {'name': 'Kraken',
    'identifier': 'kraken',
    'has_trading_incentive': False},
   'last': 0.8622,
   'volume': 70150840.26404244,
   'converted_last': {'btc': 1.585e-05, 'eth': 0.00023648, 'usd': 1.0},
   'conve